# All imports and inits

In [1]:
import gradio as gr
import os
import time
from dotenv import load_dotenv
from transformers import AutoTokenizer

# from typing import List, Tuple
from openai import OpenAI
from pinecone import Pinecone
from groq import Groq

load_dotenv()

DATA_PATH = os.getenv("DATA_PATH")
PINECONE_API = os.getenv("PINECONE_API")
# PINECONE_ENV = os.getenv("PINECONE_ENV")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
GROQ_CHAT_URL = os.getenv("GROQ_CHAT_URL")

NVIDIA_API = os.getenv("NVIDIA_API")
NVIDIA_BASE_URL = os.getenv("NVIDIA_BASE_URL")

# Configure headers for Groq API requests
GROQ_HEADERS = {
    "Authorization": f"Bearer {GROQ_API_KEY}",
    "Content-Type": "application/json",
}
# LLM_MODEL = "llama3-70b-8192"
LLM_MODEL = "llama-3.3-70b-versatile"


# NVidia Embedding import
client = OpenAI(
    api_key=NVIDIA_API,
    base_url=NVIDIA_BASE_URL,
)

"""
Input:
    - Context window: 128K
Ouput:
    - Output Max Tokens: 32,768

"""


def track_time(func):
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f"[Time Tracker] `{func.__name__}` took {end - start:.4f} seconds")
        return result

    return wrapper


# # EMBEDDING_MODEL = "llama3-405b-8192-embed"

# vo = voyageai.Client()


# Init Pinecone

In [2]:
pc = Pinecone(api_key=PINECONE_API)


# Embedding Function



In [3]:
# Connect to the index
# index = pc.Index("ai-coach")
# index = pc.Index("ahsan-400pg-pdf-doc-test")
index = pc.Index("surgical-tech-complete")  # -- COMPLETE SURGICAL TECH BOOTCAMP


# embedding_model = AutoModel.from_pretrained(
#     'jinaai/jina-embeddings-v2-base-en', trust_remote_code=True)


# # Function to generate embeddings without tokenization
# def get_embedding(data):
#     embeddings = embedding_model.encode(data).tolist()
#     return embeddings


@track_time
def get_embedding(text="None"):
    response = client.embeddings.create(
        input=text,
        model="nvidia/nv-embed-v1",
        encoding_format="float",
        extra_body={"input_type": "query", "truncate": "NONE"},
    )

    # print(response.data[0].embedding)
    # print(count_tokens(response.data[0].embedding))
    return response.data[0].embedding


get_embedding("None")


[Time Tracker] `get_embedding` took 0.6691 seconds


[-0.01506805419921875,
 -0.01165008544921875,
 0.02081298828125,
 0.0252227783203125,
 -0.0198211669921875,
 0.011077880859375,
 0.00782012939453125,
 0.037872314453125,
 -0.00841522216796875,
 0.00612640380859375,
 -0.0089569091796875,
 0.01071929931640625,
 0.00724029541015625,
 0.0088653564453125,
 0.004428863525390625,
 -0.00255584716796875,
 0.0171356201171875,
 -0.0015773773193359375,
 0.0038318634033203125,
 -0.01238250732421875,
 0.020904541015625,
 -0.00855255126953125,
 0.0176849365234375,
 0.0019550323486328125,
 -0.0012798309326171875,
 0.0123748779296875,
 -0.001995086669921875,
 0.006771087646484375,
 0.032135009765625,
 -0.00537872314453125,
 -0.02215576171875,
 0.007434844970703125,
 -0.014984130859375,
 -0.0027523040771484375,
 -0.008514404296875,
 -0.00858306884765625,
 -0.005031585693359375,
 -0.00081634521484375,
 -0.010833740234375,
 -0.0025196075439453125,
 0.008209228515625,
 -0.0005240440368652344,
 0.017333984375,
 -0.0037689208984375,
 -0.0098876953125,
 0.011

# Query Pinecone


In [4]:
# Function to query Pinecone index using embeddings
@track_time
def query_pinecone(embedding):
    # Use keyword arguments to pass the embedding and other parameters
    result = index.query(vector=embedding, top_k=5, include_metadata=True)
    return result["matches"]


print(query_pinecone(get_embedding("Pediatric surgery definition")))


[Time Tracker] `get_embedding` took 0.6780 seconds
[Time Tracker] `query_pinecone` took 1.3213 seconds
[{'id': 'vec_160',
 'metadata': {'chunk_id': 0.0,
              'file_type': 'pdf',
              'source': 'D:\\Disrupt Labs\\Rag '
                        'Experiments\\env\\Rag-pipelines-experiments\\Surgical '
                        'Technologist Bootcamp\\Course 6 - Additional Surgical '
                        'Techniques\\Lessons\\Lesson 33 - Pediatric '
                        'Surgery.pdf',
              'text': 'Pediatric Surgery\nAdditional Surgical Techniques'},
 'score': 0.715669751,
 'values': []}, {'id': 'vec_1032',
 'metadata': {'chunk_id': 0.0,
              'file_type': 'excel',
              'source': 'D:\\Disrupt Labs\\Rag '
                        'Experiments\\env\\Rag-pipelines-experiments\\Surgical '
                        'Technologist Bootcamp\\Course 6 - Additional Surgical '
                        'Techniques\\Ebooks\\Meta Data - Ebooks Course 6.xlsx',
 

# Query Groq Inference

| Use Case | Recommended top_p | Notes |
|----------|------------------|-------|
| Factual Q&A | 0.1 - 0.3 | Lower values for more deterministic, factual responses |
| Code Generation | 0.2 - 0.5 | Precision matters more than creativity |
| Technical Writing | 0.5 - 0.7 | Balanced approach for technical accuracy with clarity |
| General Conversation | 0.7 - 0.9 | Good balance for most chatbot applications |
| Creative Writing | 0.9 - 1.0 | Higher values for more diverse and creative outputs |

\n
| Parameter Combination | Use Case |
|----------------------|----------|
| top_p=0.5, temperature=0.3 | Highly factual, consistent responses |
| top_p=0.7, temperature=0.5 | Educational content with examples |
| top_p=0.9, temperature=0.7 | Creative but coherent responses |

In [5]:
# Modified query_groq function with more explicit streaming handling
@track_time
def query_groq(user_prompt, sys_prompt):
    client = Groq(api_key=os.environ["GROQ_API_KEY"])

    # Always use streaming mode
    return client.chat.completions.create(
        model=LLM_MODEL,  # or whichever model you're using
        temperature=0.5,
        messages=[
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": user_prompt},
        ],
        stream=True,
        # top_p=0.7,  # testing for better results
    )


# Print all tool calls
# print(completion.choices[0].message.executed_tools)


# Tokenizer to count number of tokens
"""
Putting tokenizer outside of the function to avoid reinitialization and optimize performance.
"""
tokenizer = AutoTokenizer.from_pretrained("jinaai/jina-embeddings-v2-base-en")


@track_time
def count_tokens(text: str) -> int:
    # Encode the text into tokens
    tokens = tokenizer.encode(text)
    return len(tokens)


# Process User Query

## Groq and Gradio with Streaming Enabled

- ### i.e. will start showing text as soon as it gets generated from groq inference
- ### faster than optimized version

## Query:
- pediatic surgery
## Response Time:
User Query Tokens: 6
[Time Tracker] `get_embedding` took 0.4752 seconds
[Time Tracker] `query_pinecone` took 0.2222 seconds
[Time Tracker] `query_groq` took 0.5060 seconds

Total time: 1.19 seconds

In [14]:
# # Modified query_groq function with more explicit streaming handling
# @track_time
# def query_groq(prompt):
#     client = Groq(api_key=os.environ["GROQ_API_KEY"])

#     # Always use streaming mode
#     return client.chat.completions.create(
#         model="llama3-70b-8192",  # or whichever model you're using
#         messages=[{"role": "user", "content": prompt}],
#         stream=True,
#     )

# --------------------------------------------------------- ## Groq and Gradio with Streaming Enabled -----------------------------------------------------
# Modified process_user_query to properly yield streaming updates
@track_time
def process_user_query(user_query: str, conversation_history: list):
    print(f"User Query Tokens: {count_tokens(user_query)}")

    # Generate embedding and get relevant context
    embedding = get_embedding(user_query)
    relevant_chunks = query_pinecone(embedding)
    context = "\n".join(chunk["metadata"]["text"] for chunk in relevant_chunks)

    # Format conversation history for the prompt
    history_str = "\n".join(
        f"User: {user}\nCoach: {response}" for user, response in conversation_history
    )
    # ---------------------------------------- OLD SINGULAR PROMPT ------------------------------------------------------------------------
    # # Create structured prompt
    # prompt = f"""You are an expert, knowledgeable, and friendly coach. Follow these guidelines carefully:

    # 1. Provide clear, step-by-step explanations to ensure deep understanding.
    # 2. Use chain-of-thought reasoning to thoroughly evaluate the provided context before responding.
    # 3. Ask guiding questions to encourage critical thinking.
    # 4. Adapt your explanation to match the student's knowledge level.
    # 5. Strictly use terminologies provided in the given context.
    # 6. Provide short, ideal examples (2-3) to illustrate your points clearly.
    # 7. Only answer based on the provided context—do not speculate or include external information.
    # 8. Always provide all specific relevant sources from the context in your responses urls, video names, video timestamps , links , resources , ebook names, lesson names , lesson numbers and anything else you think would be relevant to the user query.
    # 9. Perform sentiment analysis based on conversation history and user queries to adapt your responses empathetically and effectively.
    # 10. Must provide all relevant video timestamp from where to start watching and where to end watching
    # Context from learning materials:
    # {context}

    # Conversation history:
    # {history_str}

    # New student question:
    # "{user_query}"

    # Provide a thoughtful and contextually accurate response now:"""

    """
    
    """
    # --------------------------------------------- UPDATED SYSTEM AND USER PROMPT ------------------------------------------------------------------------
    # System prompt - contains instructions for the AI's behavior
    system_prompt = f"""
    
    Conversation history:
    {history_str}
    
    learning materials:
    {context}
    
    You are an expert, knowledgeable, and friendly coach. Follow these **guidelines** carefully:

    1. Provide clear, step-by-step explanations to ensure deep understanding.
    2. Use chain-of-thought reasoning to thoroughly evaluate the provided context before responding.
    3. Ask guiding questions to encourage critical thinking.
    4. Adapt your explanation to match the student's knowledge level.
    5. Strictly use terminologies provided in the given context.
    6. Provide short, ideal examples (2-3) to illustrate your points clearly.
    7. Only answer based on the provided learning materials—do not speculate or include external information.
    8. **Always provide all specific relevant sources with name from the context in your responses urls, video names, video timestamps, links, resources, **ebook names**, lesson names, lesson numbers, if the user query is not relevant to the context then don't provide any references and sources.**
    9. Perform sentiment analysis based on conversation history and user queries to adapt your responses empathetically and effectively.
    10. Must provide all relevant video timestamp with video name from where to start watching and where to end watching , if you can't provide timestamp then tell the user to watch the whole video.
    11. Provide a thoughtful and contextually accurate response
    12. Avoid Repetition.
    13. **If student asks something completely out of context then politely decline and ask the user to ask a question related to their course they are studying and don't provide any references and sources for this.**
    14. Please avoid "Bank Name"
    15. When users request questions, answers, quizzes or exams, generate high-quality educational assessments directly from the learning material by using for example:  multiple question types (multiple-choice with 4-5 options, true/false, short answer, fill-in-the-blanks, essay) with clear formatting; creating questions at various cognitive levels (recall, comprehension, application, analysis); providing detailed answer keys with explanations; organizing content with consistent formatting (numbered questions, lettered options, bold correct answers); adapting difficulty based on subject matter; and including step-by-step solutions for problem-solving questions.
    16. Never generate fabricated information when providing references or sources. Only provide facts, references, citations, lesson names, e-book titles, video names, timestamps explicitly present in the provided learning materials
    """

    # User prompt - contains the specific query and context
    user_prompt = f"""
    
    
    New student question:
    "{user_query}"

    """

    # Then in your query_groq function:
    stream_response = query_groq(user_prompt, system_prompt)

    # The function now directly yields the stream chunks for the Gradio interface to use
    full_response = ""

    # First, yield a response with empty text to set up the message
    # This creates the user message immediately
    temp_history = conversation_history.copy()
    temp_history.append((user_query, ""))
    yield temp_history, context

    # Process the stream
    for chunk in stream_response:
        if (
            hasattr(chunk.choices[0].delta, "content")
            and chunk.choices[0].delta.content is not None
        ):
            content_chunk = chunk.choices[0].delta.content
            full_response += content_chunk

            # Create a temporary history with the current response
            temp_history = conversation_history.copy()
            temp_history.append((user_query, full_response))

            # Yield the updated history for display
            yield temp_history, context

    # Return the final history with the complete response
    final_history = conversation_history.copy()
    final_history.append((user_query, full_response))
    yield final_history, context


@track_time
def create_gradio_interface(conversation_history):
    with gr.Blocks() as interface:
        gr.Markdown("# 🧑‍🏫 AI Coaching Assistant")
        gr.Markdown("Welcome! I'm here to help you learn. Type your question below.")

        # State management
        chat_history = gr.State(conversation_history)

        with gr.Row():
            chatbot = gr.Chatbot(height=500)
            with gr.Column(scale=0.5):
                context_display = gr.Textbox(
                    label="Relevant Context", interactive=False
                )

        user_input = gr.Textbox(label="Your Question", placeholder="Type here...")

        with gr.Row():
            submit_btn = gr.Button("Submit", variant="primary")
            undo_btn = gr.Button("Undo Last")
            clear_btn = gr.Button("Clear History")

        def handle_submit(user_query, history):
            if not user_query.strip():
                return gr.update(), history, ""

            # Use the generator directly from process_user_query
            # This will yield incremental updates as they arrive
            response_generator = process_user_query(user_query, history)

            for updated_history, context in response_generator:
                # Directly update the chatbot with each streaming chunk
                yield "", updated_history, context, updated_history

        # Component interactions with streaming support
        submit_btn.click(
            handle_submit,
            [user_input, chat_history],
            [user_input, chat_history, context_display, chatbot],
        )

        # Add submit on Enter key press
        user_input.submit(
            handle_submit,
            [user_input, chat_history],
            [user_input, chat_history, context_display, chatbot],
        )

        undo_btn.click(
            lambda history: history[:-1] if history else [],
            [chat_history],
            [chat_history],
        ).then(lambda x: x, [chat_history], [chatbot])

        clear_btn.click(lambda: [], None, [chat_history]).then(
            lambda: ([], ""), None, [chatbot, context_display]
        )

    return interface


# def main():
#     # Initialize conversation history
#     initial_conversation_history = []

#     # Create and launch the interface
#     interface = create_gradio_interface(initial_conversation_history)
#     interface.launch(share=True)


# if __name__ == "__main__":
#     main()


def main():
    """
    Main entry point for the application.

    Initializes the conversation history with a welcome message,
    creates the Gradio interface, and launches the web app.
    """
    # Initialize conversation history with welcome message
    welcome_message = "Hi there! I'm your AI coach. I can help answer questions about your course materials, explain difficult concepts, and guide your learning journey. What would you like to know today?"
    initial_conversation_history = [("", welcome_message)]

    # Create and launch the interface
    interface = create_gradio_interface(initial_conversation_history)
    interface.launch(share=True)


if __name__ == "__main__":
    main()


C:\Users\Osama\AppData\Local\Temp\ipykernel_17136\645222516.py:139: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=500)
d:\Disrupt Labs\Rag Experiments\env\Lib\site-packages\gradio\layouts\column.py:55: UserWarning: 'scale' value should be an integer. Using 0.5 will cause issues.
  warnings.warn(


[Time Tracker] `create_gradio_interface` took 0.1031 seconds
* Running on local URL:  http://127.0.0.1:7868
* Running on public URL: https://f1da8b0b9b119b5911.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Optimized Version
- ### slower than previous version
- ### slightly better responses and source citations

## Query:
- pediatic surgery
## Response Time:
User Query Tokens: 7
[Time Tracker] `get_embedding` took 0.7417 seconds
[Time Tracker] `query_pinecone` took 1.4344 seconds
[Time Tracker] `query_groq` took 0.4740 seconds

Total time = 2.64 seconds

In [ ]:
"""
-- Optimized Version ---
Streaming AI Coaching Assistant

This module implements a Gradio-based chatbot that uses streaming responses from Groq API
to provide real-time feedback to the user. It uses RAG (Retrieval-Augmented Generation)
to provide contextually relevant answers from a knowledge base.

Key components:
- query_groq: Interfaces with Groq API for streaming LLM responses
- process_user_query: Manages the streaming process and context retrieval 
- create_gradio_interface: Creates and configures the Gradio UI

The streaming implementation uses Python generators (with yield statements) to provide
real-time updates to the Gradio UI as tokens arrive from the LLM, rather than waiting
for the complete response.
"""


# Modified query_groq function with optimized streaming
# @track_time
# def query_groq(prompt):
#     """
#     Send a prompt to Groq API with streaming enabled.

#     Args:
#         prompt (str): The prompt to send to the LLM

#     Returns:
#         A stream of response chunks from the Groq API
#     """
#     client = Groq(api_key=os.environ["GROQ_API_KEY"])

#     # Always use streaming mode
#     return client.chat.completions.create(
#         model="llama3-70b-8192",  # or whichever model you're using
#         messages=[{"role": "user", "content": prompt}],
#         stream=True,
#         # Add parameters to potentially improve response time
#         temperature=0.7,
#         top_p=0.9,
#         max_tokens=2048,  # Adjust based on your needs
#     )


# Optimized process_user_query with better streaming performance
@track_time
def process_user_query(user_query: str, conversation_history: list):
    """
    Process a user query using streaming responses.

    This generator function:
    1. Retrieves relevant context from Pinecone
    2. Sends the query to Groq
    3. Yields updates as chunks arrive from the LLM

    Args:
        user_query (str): The user's question
        conversation_history (list): List of previous (query, response) tuples

    Yields:
        Tuples of (updated_history, context) as response chunks arrive
    """
    print(f"User Query Tokens: {count_tokens(user_query)}")

    # Generate embedding and get relevant context - do this once upfront
    embedding = get_embedding(user_query)
    relevant_chunks = query_pinecone(embedding)
    context = "\n".join(chunk["metadata"]["text"] for chunk in relevant_chunks)

    # Format conversation history for the prompt - do this once
    history_str = "\n".join(
        f"User: {user}\nCoach: {response}" for user, response in conversation_history
    )

    # Create structured prompt
    prompt = f"""You are an expert, knowledgeable, and friendly coach. Follow these guidelines carefully:

    1. Provide clear, step-by-step explanations to ensure deep understanding.
    2. Use chain-of-thought reasoning to thoroughly evaluate the provided context before responding.
    3. Ask guiding questions to encourage critical thinking.
    4. Adapt your explanation to match the student's knowledge level.
    5. Strictly use terminologies provided in the given context.
    6. Provide short, ideal examples (2-3) to illustrate your points clearly.
    7. Only answer based on the provided context—do not speculate or include external information.
    8. Always provide all specific relevant sources from the context in your responses urls, video names, video timestamps, links, resources, ebook names, lesson names, lesson numbers and anything else you think would be relevant to the user query.
    9. Perform sentiment analysis based on conversation history and user queries to adapt your responses empathetically and effectively.
    10. Must provide all relevant video timestamp from where to start watching and where to end watching 
    
    11. Please avoid "Bank Name" 
    Context from learning materials:
    {context}

    Conversation history:
    {history_str}

    New student question:
    "{user_query}"
    
    Provide a thoughtful and contextually accurate response now:"""

    # Get streaming LLM response
    stream_response = query_groq(prompt)

    # Initialize response
    full_response = ""

    # First, yield a response with empty text to set up the message
    # This creates the user message immediately
    temp_history = conversation_history.copy()
    temp_history.append((user_query, ""))
    yield temp_history, context

    # For efficiency, create these objects once outside the loop
    buffer = ""
    buffer_size = 10  # Characters to buffer before updating UI
    update_frequency = 0  # Counter to track updates

    # Process the stream
    for chunk in stream_response:
        if (
            hasattr(chunk.choices[0].delta, "content")
            and chunk.choices[0].delta.content is not None
        ):
            content_chunk = chunk.choices[0].delta.content
            buffer += content_chunk
            full_response += content_chunk

            # Only update UI after buffer reaches certain size or every few chunks
            # This reduces the number of updates while still maintaining responsiveness
            update_frequency += 1
            if len(buffer) >= buffer_size or update_frequency >= 5:
                temp_history = conversation_history.copy()
                temp_history.append((user_query, full_response))
                yield temp_history, context
                buffer = ""
                update_frequency = 0

    # Ensure final state is yielded if buffer has content
    if buffer or update_frequency > 0:
        final_history = conversation_history.copy()
        final_history.append((user_query, full_response))
        yield final_history, context


@track_time
def create_gradio_interface(conversation_history):
    """
    Create a Gradio interface for the coaching assistant.

    This function sets up the UI components and defines how they interact.
    The interface includes real-time streaming of LLM responses.

    Args:
        conversation_history (list): Initial conversation history

    Returns:
        gr.Blocks: The configured Gradio interface
    """
    # Welcome message to display when a new user starts
    welcome_message = "Hi there! I'm your AI learning coach. I can help answer questions about your course materials, explain difficult concepts, and guide your learning journey. What would you like to know today?"

    # Add initial welcome message to conversation history if it's empty
    if not conversation_history:
        conversation_history = [("", welcome_message)]

    with gr.Blocks() as interface:
        gr.Markdown("# 🧑‍🏫 AI Coaching Assistant")
        gr.Markdown(
            "Ask questions about your course materials and get real-time, personalized help."
        )

        # State management
        chat_history = gr.State(conversation_history)

        with gr.Row():
            chatbot = gr.Chatbot(value=conversation_history, height=500)
            with gr.Column(scale=0.5):
                context_display = gr.Textbox(
                    label="Relevant Learning Materials", interactive=False, visible=True
                )

        user_input = gr.Textbox(
            label="Your Question",
            placeholder="Type your question here and press Enter...",
            autofocus=True,
        )

        with gr.Row():
            submit_btn = gr.Button("Submit", variant="primary")
            undo_btn = gr.Button("Undo Last")
            clear_btn = gr.Button("Clear History")

        def handle_submit(user_query, history):
            """
            Handle user query submission with streaming response.

            This generator function processes the user query and yields
            updates to the UI as they become available.

            Args:
                user_query (str): The user's question
                history (list): Current conversation history

            Yields:
                Updates to the UI components
            """
            if not user_query.strip():
                return gr.update(), history, ""

            # Use the generator directly from process_user_query
            # This will yield incremental updates as they arrive
            response_generator = process_user_query(user_query, history)

            for updated_history, context in response_generator:
                # Directly update the chatbot with each streaming chunk
                yield "", updated_history, context, updated_history

        # Component interactions with streaming support
        submit_btn.click(
            handle_submit,
            [user_input, chat_history],
            [user_input, chat_history, context_display, chatbot],
        )

        # Add submit on Enter key press
        user_input.submit(
            handle_submit,
            [user_input, chat_history],
            [user_input, chat_history, context_display, chatbot],
        )

        undo_btn.click(
            lambda history: history[:-1] if history else [],
            [chat_history],
            [chat_history],
        ).then(lambda x: x, [chat_history], [chatbot])

        clear_btn.click(lambda: [("", welcome_message)], None, [chat_history]).then(
            lambda x: x, [chat_history], [chatbot]
        ).then(lambda: "", None, [context_display])

    return interface


def main():
    """
    Main entry point for the application.

    Initializes the conversation history with a welcome message,
    creates the Gradio interface, and launches the web app.
    """
    # Initialize conversation history with welcome message
    welcome_message = "Hi there! I'm your AI learning coach. I can help answer questions about your course materials, explain difficult concepts, and guide your learning journey. What would you like to know today?"
    initial_conversation_history = [("", welcome_message)]

    # Create and launch the interface
    interface = create_gradio_interface(initial_conversation_history)
    interface.launch(share=True)


if __name__ == "__main__":
    main()


: 

: 